In [1]:
#Importar bibliotecas
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
#Importar tabela
file_path = 'dados_tratados_V2.csv'
df = pd.read_csv(file_path, sep=';')
print(df)

      Age Primary streaming service  Hours per day While working  \
0    18.0                   Spotify            4.0            No   
1    61.0             YouTube Music            2.5           Yes   
2    18.0                   Spotify            4.0           Yes   
3    18.0                   Spotify            5.0           Yes   
4    18.0             YouTube Music            3.0           Yes   
..    ...                       ...            ...           ...   
607  17.0                   Spotify            2.0           Yes   
608  18.0                   Spotify            1.0           Yes   
609  19.0   Other streaming service            6.0           Yes   
610  19.0                   Spotify            5.0           Yes   
611  29.0             YouTube Music            2.0           Yes   

    Instrumentalist Composer         Fav genre Exploratory Foreign languages  \
0                No       No  Video game music          No               Yes   
1                No    

In [3]:
#Pré-processamento

# Remover a coluna 'permissions'
df = df.drop(columns=['Primary streaming service'])

# Atribuir um peso a cada resposta nas colunas de frequencia
peso_respostas = {
    'Never': 0,
    'Rarely': 1,
    'Sometimes': 2,
    'Very frequently': 3
}

# Coluna inicial e final das frequencias para aplicar o peso de cada frequencia
coluna_inicial = 'Classical_frequency'
coluna_final = 'Video game music_frequency'

# Selecionar todas as colunas entre 'coluna_inicial' e 'coluna_final'
colunas_musicas = df.loc[:, coluna_inicial:coluna_final].columns

# Troca dos valores antigos para o peso
df[colunas_musicas] = df[colunas_musicas].replace(peso_respostas)

# Mapear 'yes' para 1 e 'no' para 0
boolean_mapping = {
    'Yes': 1,
    'No': 0
}

# Lista das colunas para transformar em booleano
colunas_booleanas = ['While working', 'Instrumentalist', 'Composer', 'Exploratory', 'Foreign languages']

# Aplica o mapping
df[colunas_booleanas] = df[colunas_booleanas].replace(boolean_mapping)

# Substituir 'Improve' por 1, 'No effect' e valores vazios por 0, e remover o restante
df['Music effects'] = df['Music effects'].apply(lambda x: 1 if x == 'Improve' else (0 if x == 'No effect' or pd.isnull(x) else None))

# Remover as linhas onde o valor em 'Music effects' é None (ou seja, não é 'Improve' nem 'No effect')
df.dropna(subset=['Music effects'], inplace=True)

# Formatar colunas_booleanas para boolean
df[colunas_booleanas] = df[colunas_booleanas].astype(bool)



C:\Users\Gabriel\AppData\Local\Temp\ipykernel_4680\1149751593.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[colunas_booleanas] = df[colunas_booleanas].replace(boolean_mapping)


In [4]:
from sklearn.preprocessing import LabelEncoder

# Aplicando o LabelEncoder na coluna Fav Genre
label_encoder = LabelEncoder()
df['Fav_Genre_Encoded'] = label_encoder.fit_transform(df['Fav genre'])

df = df.drop(columns='Fav genre')





In [5]:
#Selecionar colunas para o eixo X(features)
# colunas_perfil = ['Age', 'Hours per day', 'While working','Instrumentalist', 'Composer', 'Exploratory', 'Foreign languages', 'Fav_Genre_Encoded']
colunas_transtorno = ['Anxiety', 'Depression', 'Insomnia', 'OCD']
coluna_idade = ['Age']

# colunas_x = colunas_perfil + colunas_transtorno + list(colunas_musicas)

# Trate os NaN (escolha uma das opções)
# df[colunas_x] = df[colunas_x].fillna(df[colunas_x].mean())

# Trata os NaN somente nas colunas numericas
df = df.apply(lambda x: x.fillna(x.mean()) if x.dtype in ['float64', 'int64'] else x)


In [6]:
#Desenvolvimento algoritmo

import matplotlib.pyplot as mtp  

target_column = 'Music effects'
colunas_features = list(colunas_musicas) + colunas_transtorno + coluna_idade

x = df[colunas_features].values
y = df[target_column].values

# Dividir o dataset entre treino e teste
from sklearn.model_selection import train_test_split
  
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.15, random_state = 42)  

#Padronizar o X
from sklearn.preprocessing import StandardScaler  
sc = StandardScaler()  
x_train = sc.fit_transform(x_train)  
x_test = sc.transform(x_test)  

In [7]:
from imblearn.over_sampling import SMOTE

# Aplicando SMOTE no conjunto de treino
smote = SMOTE(random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)


# Criando set de treino no Naive Bayes
from sklearn.naive_bayes import GaussianNB  
classifier =  GaussianNB()
classifier.fit(x_train_resampled, y_train_resampled) 

# Prevendo o set de resultados
y_pred = classifier.predict(x_test)  

In [8]:
#Relátorios

# Verificar precisão
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Confusion Matrix
from sklearn.metrics import confusion_matrix  
cm = confusion_matrix(y_test, y_pred)  
print(cm)

# Converter o relatório em um DataFrame
from sklearn.metrics import classification_report 

# Criar o relatório de classificação
report = classification_report(y_test, y_pred, output_dict=True)  # output_dict=True retorna um dicionário

report_df = pd.DataFrame(report).transpose() 
report_df = report_df[['precision', 'recall', 'f1-score', 'support']] 

# Exibir a tabela
print(report_df)


Accuracy: 0.4889
[[11  9]
 [37 33]]
              precision    recall  f1-score    support
0.0            0.229167  0.550000  0.323529  20.000000
1.0            0.785714  0.471429  0.589286  70.000000
accuracy       0.488889  0.488889  0.488889   0.488889
macro avg      0.507440  0.510714  0.456408  90.000000
weighted avg   0.662037  0.488889  0.530229  90.000000
